In [1]:
import sys
sys.path.append('..')
from itertools import permutations
from modified_sage_funcs import *
from imp_funcs import *

r, n = 2, 6
M = matroids.Uniform(r, n)
d = M.rank()
#S = (frozenset({0, 1, 2, 3}), frozenset({2, 3, 4, 5, 6}))
S = (frozenset({0}), frozenset({1}), frozenset({0, 2}),
    frozenset({0, 2}), frozenset({0, 3}))
S2 = (frozenset({0}), frozenset({1}), frozenset({0}))

In [2]:
def partitions(basis, weak_comp):
    parts = set()
    for perm in permutations(basis):
        partition = []
        start = 0
        for size in weak_comp:
            partition.append(frozenset(perm[start:start + size]))
            start += size
        parts.add(tuple(partition))
    return tuple(parts)

def base_partitions(bases, indexed_compositions):
    bases_partitions = {}
    for basis in bases:
        base_partitions = {}
        for i in indexed_compositions:
            base_partitions[i] = partitions(tuple(basis), indexed_compositions[i])
        bases_partitions[basis] = base_partitions
    return bases_partitions


In [3]:
def composition_weight(iwc, set_system, bp):
    weights = {}
    
    for i in iwc:
        weight = 0
        for basis in bp:
            for basis_partition in bp[basis][i]:
                flag = True
                for x, y, z in zip(iwc[i], basis_partition, set_system):
                    if len(y.intersection(z)) == x:
                        continue
                    else:
                        flag = False
                        break
                if flag:
                    weight += 1
                    break
        weights[i] = weight
    return weights

In [4]:
def weighted_poly(M, set_system):
    d, n = M.full_rank(), len(M.groundset())
    k = len(set_system)
    bases = sorted([frozenset(sorted(X)) for X in bases_iterator(M)])
    indexed_compositions = {i : x for i, x in enumerate(weak_compositions(d, k))}
    bp = base_partitions(bases, indexed_compositions)
    weights = composition_weight(indexed_compositions, set_system, bp)
    P = PolynomialRing(QQ, 'x', k)
    def monomial(wc):
        return prod([P.gen(i)**wc[i] for i in range(len(wc))])
    def weighted_sum(weights):
        return sum([weights[i]*monomial(indexed_compositions[i]) for i in weights])
    return weighted_sum(weights).factor()

weighted_poly(M, S2)

x1 * (x0 + x2)